In [1]:
import pandas as pd
from operator import itemgetter
from deepctr.models import WDL
from deepctr.feature_column import SparseFeat, get_feature_names
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('./data/avazu/ctr_data.csv', usecols = [0, 1, 11], header = 0, names = ['id', 'click', 'device_id'])

num = pd.DataFrame([x for x in range(len(set(df['id'])))])
df['item_id'] = None
df['item_id'] = num

df['user_id'] = None

l = list(set(df['device_id']))

for i in l:
    for j in range(len(df['device_id'])):
        if df['device_id'][j] == i:
            df['user_id'][j] = l.index(i)

df.to_csv('./data/avazu/ctr_data_new.csv', index = False)

F:\Ksoftware\anacondanew\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [3]:
data = pd.read_csv('./data/avazu/ctr_data_new.csv')

feature = ["item_id", "user_id"]
target = ['click']

fixlen_feature = list(SparseFeat(feat, vocabulary_size = data[feat].nunique()) for feat in feature)
linear_feature = fixlen_feature
dnn_feature = fixlen_feature

feature_name = get_feature_names(linear_feature + dnn_feature)

train_data, test_data = train_test_split(data, test_size = 0.1)

train_input = {name : train_data[name].values for name in feature_name}
test_input = {name : test_data[name].values for name in feature_name}

In [4]:
# 训练模型
model = WDL(linear_feature, dnn_feature, task = 'regression')
model.compile("adam", "mse", metrics = ['mse'])
model.fit(train_input, train_data[target].values, batch_size = 64, epochs = 10, verbose = True, validation_split = 0.1)

Train on 8100 samples, validate on 900 samples
Epoch 1/10


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


8100/8100 [==============================] - 1s 125us/sample - loss: 0.1431 - mse: 0.1431 - val_loss: 0.1353 - val_mse: 0.1353
Epoch 2/10
8100/8100 [==============================] - 0s 31us/sample - loss: 0.0565 - mse: 0.0564 - val_loss: 0.1580 - val_mse: 0.1579
Epoch 3/10
8100/8100 [==============================] - 0s 30us/sample - loss: 0.0321 - mse: 0.0320 - val_loss: 0.1417 - val_mse: 0.1416
Epoch 4/10
8100/8100 [==============================] - 0s 43us/sample - loss: 0.0219 - mse: 0.0217 - val_loss: 0.1645 - val_mse: 0.1643
Epoch 5/10
8100/8100 [==============================] - 0s 37us/sample - loss: 0.0045 - mse: 0.0043 - val_loss: 0.1381 - val_mse: 0.1380
Epoch 6/10
8100/8100 [==============================] - 0s 41us/sample - loss: 8.2694e-04 - mse: 6.7113e-04 - val_loss: 0.1551 - val_mse: 0.1549
Epoch 7/10
8100/8100 [==============================] - 0s 30us/sample - loss: 2.7609e-04 - mse: 1.2254e-04 - val_loss: 0.1476 - val_mse: 0.1474
Epoch 8/10
8100/8100 [=============

In [5]:
pred = model.predict(test_input, batch_size = 64)
mse = mean_squared_error(test_data[target].values, pred)
print("mse", mse)

mse 0.15864350666981783


In [6]:
def recommend(user, K = 10):
    pred = model.predict(test_input, batch_size = 64)
    
    item_id = test_data['id'].values
    click = test_data['click'].values
    user_id = test_data['device_id'].values
    dic = {}
    
    for i in range(0, len(user_id)):
        if click[i] < 1.0:
            dic.setdefault(user_id[i], {}).setdefault(str(item_id[i]), pred[i][0])
            
    l = list(sorted(dic[user].items(), key = itemgetter(1), reverse = True)[: K])
    ans = [x[0] for  x in l]
    
    return ans

In [7]:
rec = recommend('a99f214a')
print(rec)

['10799157821399400000', '10105320998896100000', '10819114215050300000', '10554479005249200000', '11260245336953200000', '11012310647828000000', '10307545701436500000', '10988830792029500000', '1071384169105340000', '1073457329045760000']
